## SII-GAN (Synthetical Image Inpainting GAN)

#### the generator
---
1. ResNet
2. Dilated convolution
3. 10 blocks

<center>

``` mermaid
graph TD;
    input --encode--> encode_pic;
    encode_pic --1.process--> feature;
    feature --2.decode--> feature;
    feature --> result;

```
the picture of generator, by using ResNet in 1.precess and using dilated convolation to see a larger area of the input image.

</center>

##### the BDC Block
<center>

``` mermaid
graph TD;
    x1 --> 256,3*3,r=1,64 --ReLU--> 64,1*1,r=1,16 --ReLU--> 16,3*3,r=1,16 --ReLU--> 1*1,r=1,64 --> x2;
    x1 --> 256,3*3,r=2,64 --ReLU--> 64,1*1,r=1,16 --ReLU--> 16,3*3,r=1,16 --ReLU--> 1*1,r=1,64 --> x2;
    x1 --> 256,3*3,r=4,64 --ReLU--> 64,1*1,r=1,16 --ReLU--> 16,3*3,r=1,16 --ReLU--> 1*1,r=1,64 --> x2;
    x1 --> 256,3*3,r=8,64 --ReLU--> 64,1*1,r=1,16 --ReLU--> 16,3*3,r=1,16 --ReLU--> 1*1,r=1,64 --> x2;
    x2 --+--> result;
    x1 --> 256,3*3,r=1,256 --+--> result;

```
</center>

在x1与x2叠加的过程中，在加入原图像未知的特性的情况下尽可能的保留图像已知特性。因此引入变量 $ \lambda_{gate} \in \mathbb{R}^{c \c\times h\times w}$ 由 $ x_1 \in \mathbb{R}^{c \c\times h\times w}$ 经过一次的标准卷积与sigmior操作得到，因此公式如下:

$$
    x_{result} = \lambda_{gate} \times x_1 + (1 - \lambda_{gate}) \times x_2
$$

##### encode
``` python
encoder = nn.Sequential(
            nn.Pad2D(3, mode='reflect'),
            nn.Conv2D(4, 64, 7, 1, 0),
            nn.ReLU(),
            nn.Conv2D(64, 128, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2D(128, 256, 4, 2, 1),
            nn.ReLU()
        )
```

##### decode
``` python
decoder = nn.Sequential(
            UpConv(256, 128),
            nn.ReLU(),
            UpConv(128, 64),
            nn.ReLU(),
            nn.Conv2D(64, 3, 3, 1, 1)
        )
```


#### the Discriminator
---
使用LeakyReLU对神经网络保持活力 5 层，在每一层卷积后对矩阵进行谱归一化(spectral_norm)，即：

<center>

``` mermaid
graph TD;
    inputs --> 3,3*3,s=2,64 --> spectral_norm
    LeakyReLU --> 64,3*3,s=2,128 --> spectral_norm
    LeakyReLU --> 128,3*3,s=2,256 --> spectral_norm
    LeakyReLU --> 256,3*3,s=1,512 --> spectral_norm
    LeakyReLU --> 512,s=1,1;
    spectral_norm --> LeakyReLU
    64,3*3,s=2,128 --> 128,3*3,s=2,256 --> 256,3*3,s=1,512 --> 512,s=1,1 --> output
```
</center>
这个结果会把真实的图片拉向1，假的拉向0，而不是直接置为0或者1,z(result)为：

$$
z = I \odot m + G(I \odot m, m) \odot (1 - m)
$$


#### Loss
---
##### 判别器损失
最小二乘GAN损失函数

$$
L^D_{adv} = \mathbb{E}_{z\sim p_z}[(D(z) - \sigma(m)^2)] + \mathbb{E}_{x\sim p_{data}}[(D(x) - 1)^2]
$$

##### 生成器损失
L1 Loss使用(均绝对误差（Mean Absolute Error,MAE)):
$$
L_{res}^G = \lVert G(x) - x \rVert
$$

Perceptual Loss VGG19 模型输出的各层特征图的 L1 Loss，公式如下:
$$
L_{per} = \sum_{i}\frac{\rVert \phi_{i}(x) - \phi_i(z) \lVert_1}{N}
$$

Style Loss，即使用vgg19模型:
$$
L_{sty} = \mathbb{E}_i[\lVert \phi_i(x)^T\phi_i(x) - \phi_i(z)^T\phi_i(z)\rVert_1]
$$
对抗（Adv.） Loss 即最小二乘损失:
$$
L^G_{adv} = \mathbb{E}_{z \sim p_z}[(D(z) - 1)^2]
$$


总损失
$$
L = \lambda_{adv}L_{adv}^G + \lambda_{res}L_{res} + \lambda_{per}L_{per} + \lambda_{sty}L_{sty}
$$

系数
$$
\begin{cases}
\lambda_{adv} = 0.05 \\
\lambda_{res} = 1 \\
\lambda_{per} = 0.2 \\
\lambda_{sty} = 300
\end{cases}
$$